# Notebook to manually run through the environmental chatbot

In [3]:
#!pip install openai

In [4]:
from openai import OpenAI
import json 
import time 

## Upload the Files we found to Vector Store 

In [ ]:
# enter api key here


In [1]:
# find the files to upload  
file_paths = [
    # Enter the file path here
    ""
]
import os
folder_path = "../Data/"

items = os.listdir(folder_path)

for i in items:
    file_paths.append(os.path.join(folder_path, i))
print(file_paths)
file_ids = []

for file_name in file_paths: 
    file_data = client.files.create( file=open(file_name, "rb"), purpose='assistants')

    # add file id to list of file ids
    file_ids.append(file_data.id)

['', '../Data/EPRS_BRI(2023)747880_EN.pdf', '../Data/Daily News 08 _ 02 _ 2024.html', '../Data/How To Comply With EU’s CSRD Carbon Accounting Rules.html', '../Data/European Union _ Transport Policy.html', '../Data/Light-duty vehicles _ Transport Policy.html', '../Data/EU_ Light-duty_ GHG Emissions _ Transport Policy.html', '../Data/Corporate sustainability reporting - European Commission.html', '../Data/BMUV_ The EU regulations on reducing CO2 emissions from road vehicles _ Laws.html', '../Data/Reducing carbon emissions_ EU targets and policies - EU monitor.html', '../Data/CELEX_32022L2464_EN_TXT.pdf', '../Data/The European Union leads in climate change regulations _ Wolters Kluwer.html', '../Data/Efficiency and GHG standards _ Transport Policy.html']


NameError: name 'client' is not defined

## Vectorize the File systems 

In [ ]:
vector_store = client.beta.vector_stores.create(name="travel-assistant-vectors")

# find the files to upload  
file_paths = [
    # Enter the file paths here
]

file_streams = [open(file_path, "rb") for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams
)

# Print the status and the file counts of the batch to see the result of this operation 
print(file_batch.status)

# Create the OpenAI Assistants Chatbot 

In [6]:
instructions = "You are an environment specialist. You can only talk about environmental issues, carbon emsissions, water waste, everything weather related. You can give advice. You are able to provide statistics about the environment"

In [7]:
client = OpenAI(organization="", api_key="")

assistant = client.beta.assistants.create(
  name="The Environmental Specialists Chatbot",
  instructions=instructions,
  model="gpt-3.5-turbo"
)

assistant_id = assistant.id

# Functions 
- Carbon emission calculator for businesses 

In [ ]:
function_get_carbon_emission_calculator =  {
  "name": "get_carbon_emission_percentage",
  "description": "Determine the emission of carbon of a business",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": ">>>> TODO what is the api request"
      },
      "unit": {
        "type": "string",
        "enum": [
          "c",
          "f"
        ]
      }
    },
    "required": [
      "location"
    ]
  }
}

### Handle the API Method 

In [10]:
#!pip install requests

In [1]:
# api key 
import requests

# Test1 api key
CLIMATIQ_API_KEY = 'N32NP4ZAH84S7FJ3WS3RJM0PXFC1'


In [ ]:
def get_carbon_emission_calculator(energy_kwh): 

    # Define the endpoint URL
    url = 'https://api.climatiq.io/data/v1/estimate'

    # Define the headers
    headers = {
        'Authorization': f'Bearer {CLIMATIQ_API_KEY}',
        'Content-Type': 'application/json'
    }

    # Define the data payload
    data = {
        "emission_factor": {
            "activity_id": "electricity-supply_grid-source_residual_mix",
            "data_version": "^6"
        },
        "parameters": {
            "energy": energy_kwh,
            "energy_unit": "kWh"
        }
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the response JSON
        return response.json()
    else:
        # Print the error
        print(f"Request failed with status code {response.status_code}")
        return response.text

In [ ]:
def carbon_emission_freight(vehicle, source_location, destination_location, vehicle_weight, cargo_weight):
        
    # Define the endpoint URL
    url = 'https://api.climatiq.io/freight/v1/intermodal'

    # Define the headers
    headers = {
        'Authorization': f'Bearer {CLIMATIQ_API_KEY}',
        'Content-Type': 'application/json'
    }

    # Define the data payload
    data = {
        "route": [
            { "location": { "query": source_location } },
            {
                "transport_mode": "road",# Allowed values are air, sea, road, rail
                "leg_details": {
                    "vehicle_type": vehicle,
                    "vehicle_weight": vehicle_weight
                }
            },
            { "location": { "query": destination_location } }
        ],
        "cargo": {
            "weight": int(cargo_weight),
            "weight_unit": "t"
        }
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the response JSON
        return str(response.json())
    else:
        # Print the error
        print(f"Request failed with status code {response.status_code}")
        return response.text


#### Update the Assistants API with these functions 

In [ ]:
# update the assistants api agent to have these new methods 
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  instructions=instructions,
  tools = [
    {"type":"file_search"},
    {"type":"function", "function": get_carbon_emission_calculator}
    ]
)

## Machine Learning Model ( Functions Addition )

## Code Interpreter for Graph / Report Generator 